As of MLflow 2.9.0, Model Stages have been deprecated and will be removed in a future major release (https://www.mlflow.org/docs/latest/model-registry.html#migrating-from-stages)
Summary: Stages are rpaced with aliases now. It means that instead of "models:/regression_model/Production" there should be something like: "models:/prod.ml_team.regression_model@champion"